In [1]:
import csv
import pandas as pd
from bs4 import BeautifulSoup
import requests
import time
import re

In [2]:
df = pd.read_csv('./googleplaystore.csv', engine="python")
df.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


In [6]:
from pymongo import MongoClient
client = MongoClient('localhost', 27017)
db = client.apps
collection = db.app_name
print(collection)

# Additional apps to be extracted #
apps_list = ['Amazon','eBay','Flipkart','Snapdeal','Myntra','Jabong','HomeShop18','berniaga.com','PayPal','Etsy','Wanelo Shopping','Wish - Shopping Made Fun','AliExpress','Alibaba.com','Big Sale In Aliexpress','AliExpress Super Deals','ZZKKO - Shop for Less','Shopping at DealExtreme','DinoDirect - Shopping China','DHgate Mobile online wholesale','Shopping World','DHgate Mobile','Forever 21','ZOVI','ASOS','River Island','OLX Free Classifieds','Quikr Free Classifieds','Polyvore: Shop, Style, Fashion','LivingSocial','Groupon - Daily Deals, Coupons','The Coupons App','Coupons.com Coupons & Codes','Extreme Coupon Finder','RetailMeNot Coupons','COUPONS & WEEKLY ADS -Shopular']
df_apps = pd.DataFrame({'App': apps_list})

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'apps'), 'app_name')
                               App
0                           Amazon
1                             eBay
2                         Flipkart
3                         Snapdeal
4                           Myntra
5                           Jabong
6                       HomeShop18
7                     berniaga.com
8                           PayPal
9                             Etsy
10                 Wanelo Shopping
11        Wish - Shopping Made Fun
12                      AliExpress
13                     Alibaba.com
14          Big Sale In Aliexpress
15          AliExpress Super Deals
16           ZZKKO - Shop for Less
17         Shopping at DealExtreme
18     DinoDirect - Shopping China
19  DHgate Mobile online wholesale
20                  Shopping World
21                   DHgate Mobile
22                      Forever 21
23                           

# Executing Extraction of description of the apps

In [4]:
for index, row in df.iterrows():
#for index, row in df_apps.iterrows():
    print(index)
    print(row['App'])
    
    
    app_name = str(row['App']).replace(' ', '%20')
    url_1 = 'https://play.google.com/store/search?q={0}&c=apps&hl=en'.format(app_name)
    p = requests.get(url_1);
    soup = BeautifulSoup(p.text, 'html.parser')
    val = soup.find('a', href= re.compile(r'^\/store\/apps\/details.*'))

    url_2 = val['href']
    url_2 = 'https://play.google.com' + url_2


    time.sleep(2)

    r = requests.get(url_2);
    soup = BeautifulSoup(r.text, 'html.parser')

    ## app category ##
    category = soup.find(itemprop="genre")
    category = category.text
    print(category)

    ## app description ##
    artist_name_list = soup.find(itemprop="description")
    description = artist_name_list.get('content')

    input_json = {'app_name': row['App'], 'description': description, 'category': category, 'updated_at': time.time()}
    collection.update({'app_name': row['App']}, input_json, upsert = True)
    if index == 1000:
        break


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

/Users/gaurav/anaconda2/envs/similarapps/lib/python3.6/site-packages/ipykernel_launcher.py:42: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.


/store/apps/details?id=com.zombodroid.MemeGenerator
https://play.google.com/store/apps/details?id=com.zombodroid.MemeGenerator
Entertainment
1002
SketchBook - draw and paint
https://play.google.com/store/search?q=SketchBook%20-%20draw%20and%20paint&c=apps&hl=en
/store/apps/details?id=com.adsk.sketchbook
https://play.google.com/store/apps/details?id=com.adsk.sketchbook
Entertainment
1003
Colorfy: Coloring Book for Adults - Free
https://play.google.com/store/search?q=Colorfy:%20Coloring%20Book%20for%20Adults%20-%20Free&c=apps&hl=en
/store/apps/details?id=com.fungamesforfree.colorfy
https://play.google.com/store/apps/details?id=com.fungamesforfree.colorfy
Entertainment
1004
All Events in City
https://play.google.com/store/search?q=All%20Events%20in%20City&c=apps&hl=en
/store/apps/details?id=com.amitech.allevents
https://play.google.com/store/apps/details?id=com.amitech.allevents
Events
1005
Ticketmaster Event Tickets
https://play.google.com/store/search?q=Ticketmaster%20Event%20Tickets&c=

/store/apps/details?id=com.fmworld.messenger
https://play.google.com/store/apps/details?id=com.fmworld.messenger
Events
1035
Goldstar: Live Event Tickets
https://play.google.com/store/search?q=Goldstar:%20Live%20Event%20Tickets&c=apps&hl=en
/store/apps/details?id=com.goldstar
https://play.google.com/store/apps/details?id=com.goldstar
Events
1036
Emmabodafestivalen
https://play.google.com/store/search?q=Emmabodafestivalen&c=apps&hl=en
/store/apps/details?id=se.emmabodafestivalen.app
https://play.google.com/store/apps/details?id=se.emmabodafestivalen.app
Events
1037
KudaGo - things to do in NY
https://play.google.com/store/search?q=KudaGo%20-%20things%20to%20do%20in%20NY&c=apps&hl=en
/store/apps/details?id=com.kudago.android
https://play.google.com/store/apps/details?id=com.kudago.android
Events
1038
Mindvalley U Tallinn 2018
https://play.google.com/store/search?q=Mindvalley%20U%20Tallinn%202018&c=apps&hl=en
/store/apps/details?id=com.mindvalley.mva
https://play.google.com/store/apps/det

Finance
1072
Banorte Movil
https://play.google.com/store/search?q=Banorte%20Movil&c=apps&hl=en
/store/apps/details?id=org.microemu.android.model.common.VTUserApplicationBNRTMB
https://play.google.com/store/apps/details?id=org.microemu.android.model.common.VTUserApplicationBNRTMB
Finance
1073
Wells Fargo Mobile
https://play.google.com/store/search?q=Wells%20Fargo%20Mobile&c=apps&hl=en
/store/apps/details?id=com.wf.wellsfargomobile
https://play.google.com/store/apps/details?id=com.wf.wellsfargomobile
Finance
1074
Credit Karma
https://play.google.com/store/search?q=Credit%20Karma&c=apps&hl=en
/store/apps/details?id=com.creditkarma.mobile
https://play.google.com/store/apps/details?id=com.creditkarma.mobile
Finance
1075
BZWBK24 mobile
https://play.google.com/store/search?q=BZWBK24%20mobile&c=apps&hl=en
/store/apps/details?id=pl.bzwbk.bzwbk24
https://play.google.com/store/apps/details?id=pl.bzwbk.bzwbk24
Finance
1076
PayPal
https://play.google.com/store/search?q=PayPal&c=apps&hl=en
/store/ap

/store/apps/details?id=com.movencorp.moven
https://play.google.com/store/apps/details?id=com.movencorp.moven
Finance
1108
Qapital - Save Small. Live Large
https://play.google.com/store/search?q=Qapital%20-%20Save%20Small.%20Live%20Large&c=apps&hl=en
/store/apps/details?id=com.qapital
https://play.google.com/store/apps/details?id=com.qapital
Finance
1109
Experian - Free Credit Report
https://play.google.com/store/search?q=Experian%20-%20Free%20Credit%20Report&c=apps&hl=en
/store/apps/details?id=com.experian.android
https://play.google.com/store/apps/details?id=com.experian.android
Finance
1110
Branch
https://play.google.com/store/search?q=Branch&c=apps&hl=en
/store/apps/details?id=com.branchmessenger
https://play.google.com/store/apps/details?id=com.branchmessenger
Finance
1111
WalletHub - Free Credit Score, Report & Monitoring
https://play.google.com/store/search?q=WalletHub%20-%20Free%20Credit%20Score,%20Report%20&%20Monitoring&c=apps&hl=en
/store/apps/details?id=com.wallethub.mywalle

/store/apps/details?id=org.yccheok.jstock.gui
https://play.google.com/store/apps/details?id=org.yccheok.jstock.gui
Finance
1142
Digit Save Money Automatically
https://play.google.com/store/search?q=Digit%20Save%20Money%20Automatically&c=apps&hl=en
/store/apps/details?id=digit.android
https://play.google.com/store/apps/details?id=digit.android
Finance
1143
Acorns - Invest Spare Change
https://play.google.com/store/search?q=Acorns%20-%20Invest%20Spare%20Change&c=apps&hl=en
/store/apps/details?id=com.acorns.android
https://play.google.com/store/apps/details?id=com.acorns.android
Finance
1144
Webull - Stock Quotes & Free Stock Trading
https://play.google.com/store/search?q=Webull%20-%20Stock%20Quotes%20&%20Free%20Stock%20Trading&c=apps&hl=en
/store/apps/details?id=org.dayup.stocks
https://play.google.com/store/apps/details?id=org.dayup.stocks
Finance
1145
Trading 212 - Forex, Stocks, CFDs
https://play.google.com/store/search?q=Trading%20212%20-%20Forex,%20Stocks,%20CFDs&c=apps&hl=en
/store